In [13]:
library(tidyverse)
library(magrittr)
library(reshape2)
library(ggplot2)
library(scales)
library(ggrepel)
library(readxl)
library(corrplot)
library(purrr)
library(rlang)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v tibble  2.1.1     v purrr   0.3.2
v readr   1.3.1     v stringr 1.4.0
v tibble  2.1.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x readr::col_factor() masks scales::col_factor()
x purrr::discard()    masks scales::discard()
x dplyr::filter()     masks stats::filter()
x dplyr::lag()        masks stats::lag()

Attaching package: 'magrittr'

The following object is masked from 'package:purrr':

    set_names

The following object is masked from 'package:tidyr':

    extract



# EDA

# Importing The Data

In [124]:
input_path <- "D:/OneDrive - UTS/36103/AT2B/EDA"
output_path <- "D:/OneDrive - UTS/36103/AT2B/data"

In [38]:
input_list < list.files(input_path)
input_list

[[1]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/age.csv"

[[2]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/country_of_birth.csv"

[[3]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/crime.csv"

[[4]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/industry_employment.csv"

[[5]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/labour.csv"

[[6]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/pop_dens.csv"

[[7]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/seifa.csv"

[[8]]
[1] "D:/OneDrive - UTS/36103/AT2B/EDA/transport.csv"

In [35]:
tbl_names <- map(input_list, function(x) strsplit(x, "\\.")[[1]][[1]])
tbl_names

[[1]]
[1] "age"

[[2]]
[1] "country_of_birth"

[[3]]
[1] "crime"

[[4]]
[1] "industry_employment"

[[5]]
[1] "labour"

[[6]]
[1] "pop_dens"

[[7]]
[1] "seifa"

[[8]]
[1] "transport"

### Column Definitions

In [86]:
cols_list <- list(age = cols(Age = col_factor(),
                             Sex = col_factor(),
                             `Labour force status` = col_factor(),
                             LGA = col_factor(),
                             Value = col_integer()),
                  country_of_birth = cols(birthplace_of_person = col_factor(),
                                          LGA = col_factor(),
                                          Value = col_integer(),
                                          year_of_arrival_in_australia = col_factor()),
                  crime = cols(LGA = col_factor(),
                               Offence = col_factor(),
                               Crim_Count = col_integer()),
                  industry_employment = cols(LGA = col_factor(),
                                             Value = col_integer(),
                                             Age = col_factor(),
                                             Sex = col_factor(),
                                             `Industry of Employment` = col_factor()),
                  labour = cols(`Labour force status` = col_factor(),
                                LGA = col_factor(),
                                Value = col_integer(),
                                Age = col_factor(),
                                Sex = col_factor()),
                  pop_dens = cols(LGA = col_factor(),
                                  Pop_Dens_km2 = col_double()),
                  seifa = cols(LGA = col_factor(),
                               Value = col_integer()),
                  seifa = cols(LGA = col_factor(),
                               route_type = col_factor(),
                               num_service = col_double()))

In [87]:
input_data <- map2(input_list, cols_list, ~readr::read_csv(.x, col_types = .y))

Warning message:
"The following named parsers don't match the column names: Value"Warning message:
"Missing column names filled in: 'X1' [1]"Warning message:
"Missing column names filled in: 'X1' [1]"Warning message:
"Missing column names filled in: 'X1' [1]"

In [88]:
names(input_data) = tbl_names
names(input_data)

[1] "age"                 "country_of_birth"    "crime"              
[4] "industry_employment" "labour"              "pop_dens"           
[7] "seifa"               "transport"

In [89]:
map(input_data, head)

$age
# A tibble: 6 x 8
  Age    Sex    `Labour force sta~ State   LGA_2016 LGA      `Census year` Value
  <fct>  <fct>  <fct>              <chr>      <dbl> <fct>            <dbl> <int>
1 15 - ~ Males  Employed, worked ~ New So~    10050 Albury            2016   236
2 15 - ~ Femal~ Employed, worked ~ New So~    10050 Albury            2016   113
3 15 - ~ Perso~ Employed, worked ~ New So~    10050 Albury            2016   348
4 15 - ~ Males  Employed, worked ~ New So~    10130 Armidal~          2016   105
5 15 - ~ Femal~ Employed, worked ~ New So~    10130 Armidal~          2016    58
6 15 - ~ Perso~ Employed, worked ~ New So~    10130 Armidal~          2016   160

$country_of_birth
# A tibble: 6 x 7
  birthplace_of_pe~ census_year lga_2016 LGA   value yarrp_2016 year_of_arrival~
  <fct>                   <dbl>    <dbl> <fct> <dbl> <chr>      <fct>           
1 Oceania and Anta~        2016    10050 Albu~     0 1          Arrived 1900 - ~
2 Oceania and Anta~        2016    10050 Albu~     0 2          Arrived 1946 - ~
3 Oceania and Anta~        2016    10050 Albu~     0 3          Arrived 1956 - ~
4 Oceania and Anta~        2016    10050 Albu~     0 4          Arrived 1966 - ~
5 Oceania and Anta~        2016    10050 Albu~     0 5          Arrived 1976 - ~
6 Oceania and Anta~        2016    10050 Albu~     0 6          Arrived 1986 - ~

$crime
# A tibble: 6 x 4
     X1 LGA    Offence                               Crim_Count
  <dbl> <fct>  <fct>                                      <int>
1     1 Albury Murder *                                       2
2     2 Albury Attempted murder                               1
3     3 Albury Murder accessory, conspiracy                   0
4     4 Albury Manslaughter *                                 0
5     5 Albury Domestic violence related assault            284
6     6 Albury Non-domestic violence related assault        296

$industry_employment
# A tibble: 6 x 7
  Age    Sex    `Industry of Employment` LGA_2016 LGA        `Census year` Value
  <fct>  <fct>  <fct>                       <dbl> <fct>              <dbl> <int>
1 15 - ~ Males  Agriculture, Forestry a~    10050 Albury              2016     6
2 15 - ~ Femal~ Agriculture, Forestry a~    10050 Albury              2016     4
3 15 - ~ Perso~ Agriculture, Forestry a~    10050 Albury              2016     6
4 15 - ~ Males  Agriculture, Forestry a~    10130 Armidale ~          2016    34
5 15 - ~ Femal~ Agriculture, Forestry a~    10130 Armidale ~          2016     9
6 15 - ~ Perso~ Agriculture, Forestry a~    10130 Armidale ~          2016    43

$labour
# A tibble: 6 x 7
  Age     Sex     `Labour force status` LGA_2016 LGA         `Census year` Value
  <fct>   <fct>   <fct>                    <dbl> <fct>               <dbl> <int>
1 15 - 19 Males   Employed, worked ful~    10050 Albury               2016   236
2 15 - 19 Females Employed, worked ful~    10050 Albury               2016   113
3 15 - 19 Persons Employed, worked ful~    10050 Albury               2016   348
4 15 - 19 Males   Employed, worked ful~    10130 Armidale R~          2016   105
5 15 - 19 Females Employed, worked ful~    10130 Armidale R~          2016    58
6 15 - 19 Persons Employed, worked ful~    10130 Armidale R~          2016   160

$pop_dens
# A tibble: 6 x 3
     X1 LGA               Pop_Dens_km2
  <dbl> <fct>                    <dbl>
1     1 Albury                   170. 
2     2 Armidale Regional          3.5
3     3 Ballina                   88.7
4     4 Balranald                  0.1
5     5 Bathurst Regional         11.1
6     6 Bega Valley                5.4

$seifa
# A tibble: 6 x 3
     X1 LGA               Value
  <dbl> <fct>             <int>
1     1 Albury              964
2     2 Armidale Dumaresq   954
3     3 Ashfield            962
4     4 Auburn              931
5     5 Ballina             986
6     6 Balranald           969

$transport
# A tibble: 6 x 3
  LGA               route_type num_service
  <fct>             <fct>            <dbl>
1 Al

In [92]:
for(i in 1:length(input_data)){
    print(names(input_data)[[i]])
    print(glimpse(input_data[[i]]))
}

[1] "age"
Observations: 42,768
Variables: 8
$ Age                   <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 19, ...
$ Sex                   <fct> Males, Females, Persons, Males, Females, Pers...
$ `Labour force status` <fct> "Employed, worked full-time", "Employed, work...
$ State                 <chr> "New South Wales", "New South Wales", "New So...
$ LGA_2016              <dbl> 10050, 10050, 10050, 10130, 10130, 10130, 102...
$ LGA                   <fct> Albury, Albury, Albury, Armidale Regional, Ar...
$ `Census year`         <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...
$ Value                 <int> 236, 113, 348, 105, 58, 160, 115, 53, 168, 11...
# A tibble: 42,768 x 8
   Age    Sex    `Labour force sta~ State   LGA_2016 LGA     `Census year` Value
   <fct>  <fct>  <fct>              <chr>      <dbl> <fct>           <dbl> <int>
 1 15 - ~ Males  Employed, worked ~ New So~    10050 Albury           2016   236
 2 15 - ~ Femal~ Employed, worked ~ New So~    10050 Albur

## Reducing Size

The `country_of_birth` data set contains many zero values. The following step removes these rows.

### Remove Zero Values

In [75]:
head(input_data$country_of_birth)

birthplace_of_person,census_year,lga_2016,LGA,value,yarrp_2016,year_of_arrival_in_australia
"Oceania and Antarctica, nfd",2016,10050,Albury,0,1,Arrived 1900 - 1945
"Oceania and Antarctica, nfd",2016,10050,Albury,0,2,Arrived 1946 - 1955
"Oceania and Antarctica, nfd",2016,10050,Albury,0,3,Arrived 1956 - 1965
"Oceania and Antarctica, nfd",2016,10050,Albury,0,4,Arrived 1966 - 1975
"Oceania and Antarctica, nfd",2016,10050,Albury,0,5,Arrived 1976 - 1985
"Oceania and Antarctica, nfd",2016,10050,Albury,0,6,Arrived 1986 - 1995


In [93]:
input_data$country_of_birth %>%
    filter(value == 0) %>%
    dim

[1] 810442      7

In [94]:
input_data$country_of_birth %<>%
    filter(value != 0)

head(input_data$country_of_birth)
glimpse(input_data$country_of_birth)

birthplace_of_person,census_year,lga_2016,LGA,value,yarrp_2016,year_of_arrival_in_australia
New Zealand,2016,10050,Albury,7,2,Arrived 1946 - 1955
New Zealand,2016,10050,Albury,22,3,Arrived 1956 - 1965
New Zealand,2016,10050,Albury,63,4,Arrived 1966 - 1975
New Zealand,2016,10050,Albury,113,5,Arrived 1976 - 1985
New Zealand,2016,10050,Albury,92,6,Arrived 1986 - 1995
New Zealand,2016,10050,Albury,87,7,Arrived 1996 - 2005


Observations: 108,163
Variables: 7
$ birthplace_of_person         <fct> New Zealand, New Zealand, New Zealand,...
$ census_year                  <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 20...
$ lga_2016                     <dbl> 10050, 10050, 10050, 10050, 10050, 100...
$ LGA                          <fct> Albury, Albury, Albury, Albury, Albury...
$ value                        <dbl> 7, 22, 63, 113, 92, 87, 6, 24, 7, 12, ...
$ yarrp_2016                   <chr> "2", "3", "4", "5", "6", "7", "9", "Z"...
$ year_of_arrival_in_australia <fct> Arrived 1946 - 1955, Arrived 1956 - 19...


# Joining the Data Sets Together

## Find the Data Set With The Most Complete List of LGAs

In [95]:
input_data %>% map(function(tbl) tbl['LGA'] %>% unique %>% dim %>% extract(1))

$age
[1] 132

$country_of_birth
[1] 132

$crime
[1] 130

$industry_employment
[1] 132

$labour
[1] 132

$pop_dens
[1] 129

$seifa
[1] 556

$transport
[1] 128

Seems like `seifa` has the most number of LGAs, but we know that there are only about 130 LGAs in NSW. So maybe the LGAs in the `seifa` data includes non-NSW LGAs too. A glance of the tail of the `seifa` data shows that there are indeed non-NSW areas, e.g. `Unincorporated ACT`.

We will not use the `seifa` data as the starting point, instead `country_of_birth` will be used for `left_join`ing the other data sets, i.e. LGAs that are not in the `country_of_birth` data set will not be joined.

In [96]:
input_data$seifa %>% 
    tail

X1,LGA,Value
559,Tiwi Islands,589
560,Victoria-Daly,613
561,Wagait,1011
562,West Arnhem,640
563,Unincorporated NT,966
564,Unincorporated ACT,1051


## Join `country_of_birth` with `age`

Ignore the `year_of_arrival_in_australia` in the data set `country_of_birth`, because it's too big to be joined and will throw `cannot allocate vector of size 133.7 Mb`.

In [104]:
joined <- input_data$country_of_birth %>%
    group_by(LGA, birthplace_of_person) %>%
    summarise(births = sum(value)) %>%
    left_join(input_data$age, by = "LGA")

head(joined)
glimpse(joined)

LGA,birthplace_of_person,births,Age,Sex,Labour force status,State,LGA_2016,Census year,Value
Albury,New Zealand,2510,15 - 19,Males,"Employed, worked full-time",New South Wales,10050,2016,236
Albury,New Zealand,2510,15 - 19,Females,"Employed, worked full-time",New South Wales,10050,2016,113
Albury,New Zealand,2510,15 - 19,Persons,"Employed, worked full-time",New South Wales,10050,2016,348
Albury,New Zealand,2510,15 - 19,Males,"Employed, worked part-time",New South Wales,10050,2016,431
Albury,New Zealand,2510,15 - 19,Females,"Employed, worked part-time",New South Wales,10050,2016,641
Albury,New Zealand,2510,15 - 19,Persons,"Employed, worked part-time",New South Wales,10050,2016,1074


Observations: 2,756,592
Variables: 10
Groups: LGA [132]
$ LGA                   <fct> Albury, Albury, Albury, Albury, Albury, Albur...
$ birthplace_of_person  <fct> New Zealand, New Zealand, New Zealand, New Ze...
$ births                <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, 251...
$ Age                   <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 19, ...
$ Sex                   <fct> Males, Females, Persons, Males, Females, Pers...
$ `Labour force status` <fct> "Employed, worked full-time", "Employed, work...
$ State                 <chr> "New South Wales", "New South Wales", "New So...
$ LGA_2016              <dbl> 10050, 10050, 10050, 10050, 10050, 10050, 100...
$ `Census year`         <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 201...
$ Value                 <int> 236, 113, 348, 431, 641, 1074, 27, 26, 59, 22...


In [121]:
joined <- input_data$country_of_birth %>%
    group_by(LGA, birthplace_of_person) %>%
    summarise(num_births = sum(value)) %>%
    left_join(input_data$age) %>%
    select(-State, -LGA_2016, -`Census year`) %>%
    rename(num_emp = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"


LGA,birthplace_of_person,num_births,Age,Sex,Labour force status,num_emp
Albury,New Zealand,2510,15 - 19,Males,"Employed, worked full-time",236
Albury,New Zealand,2510,15 - 19,Females,"Employed, worked full-time",113
Albury,New Zealand,2510,15 - 19,Persons,"Employed, worked full-time",348
Albury,New Zealand,2510,15 - 19,Males,"Employed, worked part-time",431
Albury,New Zealand,2510,15 - 19,Females,"Employed, worked part-time",641
Albury,New Zealand,2510,15 - 19,Persons,"Employed, worked part-time",1074


Observations: 2,756,592
Variables: 7
Groups: LGA [132]
$ LGA                   <fct> Albury, Albury, Albury, Albury, Albury, Albur...
$ birthplace_of_person  <fct> New Zealand, New Zealand, New Zealand, New Ze...
$ num_births            <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, 251...
$ Age                   <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 19, ...
$ Sex                   <fct> Males, Females, Persons, Males, Females, Pers...
$ `Labour force status` <fct> "Employed, worked full-time", "Employed, work...
$ num_emp               <int> 236, 113, 348, 431, 641, 1074, 27, 26, 59, 22...


## Add `crime`

In [123]:
joined %>% left_join(input_data$crime) %>%
    head(20)


Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

ERROR: Error: Evaluation error: cannot allocate vector of size 618.7 Mb.


Seems like three joined data sets is too big. So the following joins contains the dependent variable `crime`.

# Joining `crime` and `age`

In [179]:
head(input_data$crime)
head(input_data$age)

X1,LGA,Offence,Crim_Count
1,Albury,Murder *,2
2,Albury,Attempted murder,1
3,Albury,"Murder accessory, conspiracy",0
4,Albury,Manslaughter *,0
5,Albury,Domestic violence related assault,284
6,Albury,Non-domestic violence related assault,296


Age,Sex,Labour force status,State,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Employed, worked full-time",New South Wales,10050,Albury,2016,236
15 - 19,Females,"Employed, worked full-time",New South Wales,10050,Albury,2016,113
15 - 19,Persons,"Employed, worked full-time",New South Wales,10050,Albury,2016,348
15 - 19,Males,"Employed, worked full-time",New South Wales,10130,Armidale Regional,2016,105
15 - 19,Females,"Employed, worked full-time",New South Wales,10130,Armidale Regional,2016,58
15 - 19,Persons,"Employed, worked full-time",New South Wales,10130,Armidale Regional,2016,160


In [180]:
joined <- input_data$crime %>%
    select(-X1) %>%
    left_join(input_data$age %>% select(-State, -LGA_2016, -`Census year`)) %>%
    rename(pop_count = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,Offence,Crim_Count,Age,Sex,Labour force status,pop_count
Albury,Murder *,2,15 - 19,Males,"Employed, worked full-time",236
Albury,Murder *,2,15 - 19,Females,"Employed, worked full-time",113
Albury,Murder *,2,15 - 19,Persons,"Employed, worked full-time",348
Albury,Murder *,2,15 - 19,Males,"Employed, worked part-time",431
Albury,Murder *,2,15 - 19,Females,"Employed, worked part-time",641
Albury,Murder *,2,15 - 19,Persons,"Employed, worked part-time",1074


Observations: 2,511,310
Variables: 7
$ LGA                   <chr> "Albury", "Albury", "Albury", "Albury", "Albu...
$ Offence               <fct> Murder *, Murder *, Murder *, Murder *, Murde...
$ Crim_Count            <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
$ Age                   <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 19, ...
$ Sex                   <fct> Males, Females, Persons, Males, Females, Pers...
$ `Labour force status` <fct> "Employed, worked full-time", "Employed, work...
$ pop_count             <int> 236, 113, 348, 431, 641, 1074, 27, 26, 59, 22...


In [181]:
out_filename = "joined_crime_age.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `country_of_birth`

In [176]:
head(input_data$crime)
head(input_data$country_of_birth)

X1,LGA,Offence,Crim_Count
1,Albury,Murder *,2
2,Albury,Attempted murder,1
3,Albury,"Murder accessory, conspiracy",0
4,Albury,Manslaughter *,0
5,Albury,Domestic violence related assault,284
6,Albury,Non-domestic violence related assault,296


birthplace_of_person,census_year,lga_2016,LGA,value,yarrp_2016,year_of_arrival_in_australia
New Zealand,2016,10050,Albury,7,2,Arrived 1946 - 1955
New Zealand,2016,10050,Albury,22,3,Arrived 1956 - 1965
New Zealand,2016,10050,Albury,63,4,Arrived 1966 - 1975
New Zealand,2016,10050,Albury,113,5,Arrived 1976 - 1985
New Zealand,2016,10050,Albury,92,6,Arrived 1986 - 1995
New Zealand,2016,10050,Albury,87,7,Arrived 1996 - 2005


In [177]:
joined <- input_data$crime %>%
    select(-X1) %>%
    left_join(input_data$country_of_birth %>% select(-lga_2016, -census_year, -yarrp_2016)) %>%
    rename(pop_count = value)

head(joined)
glimpse(joined)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,Offence,Crim_Count,birthplace_of_person,pop_count,year_of_arrival_in_australia
Albury,Murder *,2,New Zealand,7,Arrived 1946 - 1955
Albury,Murder *,2,New Zealand,22,Arrived 1956 - 1965
Albury,Murder *,2,New Zealand,63,Arrived 1966 - 1975
Albury,Murder *,2,New Zealand,113,Arrived 1976 - 1985
Albury,Murder *,2,New Zealand,92,Arrived 1986 - 1995
Albury,Murder *,2,New Zealand,87,Arrived 1996 - 2005


Observations: 6,382,714
Variables: 6
$ LGA                          <chr> "Albury", "Albury", "Albury", "Albury"...
$ Offence                      <fct> Murder *, Murder *, Murder *, Murder *...
$ Crim_Count                   <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,...
$ birthplace_of_person         <fct> New Zealand, New Zealand, New Zealand,...
$ pop_count                    <dbl> 7, 22, 63, 113, 92, 87, 6, 24, 3, 9, 6...
$ year_of_arrival_in_australia <fct> Arrived 1946 - 1955, Arrived 1956 - 19...


In [178]:
out_filename = "joined_crime_countryofbirth.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `industry_employment`

In [173]:
head(input_data$crime)
head(input_data$industry_employment)

X1,LGA,Offence,Crim_Count
1,Albury,Murder *,2
2,Albury,Attempted murder,1
3,Albury,"Murder accessory, conspiracy",0
4,Albury,Manslaughter *,0
5,Albury,Domestic violence related assault,284
6,Albury,Non-domestic violence related assault,296


Age,Sex,Industry of Employment,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Agriculture, Forestry and Fishing",10050,Albury,2016,6
15 - 19,Females,"Agriculture, Forestry and Fishing",10050,Albury,2016,4
15 - 19,Persons,"Agriculture, Forestry and Fishing",10050,Albury,2016,6
15 - 19,Males,"Agriculture, Forestry and Fishing",10130,Armidale Regional,2016,34
15 - 19,Females,"Agriculture, Forestry and Fishing",10130,Armidale Regional,2016,9
15 - 19,Persons,"Agriculture, Forestry and Fishing",10130,Armidale Regional,2016,43


In [174]:
joined <- input_data$crime %>%
    select(-X1) %>%
    left_join(input_data$industry_employment %>% select(-LGA_2016, -`Census year`)) %>%
    rename(pop_count = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,Offence,Crim_Count,Age,Sex,Industry of Employment,pop_count
Albury,Murder *,2,15 - 19,Males,"Agriculture, Forestry and Fishing",6
Albury,Murder *,2,15 - 19,Females,"Agriculture, Forestry and Fishing",4
Albury,Murder *,2,15 - 19,Persons,"Agriculture, Forestry and Fishing",6
Albury,Murder *,2,15 - 19,Males,Mining,0
Albury,Murder *,2,15 - 19,Females,Mining,0
Albury,Murder *,2,15 - 19,Persons,Mining,0


Observations: 4,650,310
Variables: 7
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ Offence                  <fct> Murder *, Murder *, Murder *, Murder *, Mu...
$ Crim_Count               <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
$ Age                      <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 1...
$ Sex                      <fct> Males, Females, Persons, Males, Females, P...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ pop_count                <int> 6, 4, 6, 0, 0, 0, 49, 44, 90, 0, 0, 0, 121...


In [175]:
out_filename = "joined_crime_employment.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `labour`

In [170]:
head(input_data$crime)
head(input_data$labour)

X1,LGA,Offence,Crim_Count
1,Albury,Murder *,2
2,Albury,Attempted murder,1
3,Albury,"Murder accessory, conspiracy",0
4,Albury,Manslaughter *,0
5,Albury,Domestic violence related assault,284
6,Albury,Non-domestic violence related assault,296


Age,Sex,Labour force status,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Employed, worked full-time",10050,Albury,2016,236
15 - 19,Females,"Employed, worked full-time",10050,Albury,2016,113
15 - 19,Persons,"Employed, worked full-time",10050,Albury,2016,348
15 - 19,Males,"Employed, worked full-time",10130,Armidale Regional,2016,105
15 - 19,Females,"Employed, worked full-time",10130,Armidale Regional,2016,58
15 - 19,Persons,"Employed, worked full-time",10130,Armidale Regional,2016,160


In [171]:
joined <- input_data$crime %>%
    select(-X1) %>%
    left_join(input_data$labour %>% select(-LGA_2016, -`Census year`)) %>%
    rename(pop_count = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,Offence,Crim_Count,Age,Sex,Labour force status,pop_count
Albury,Murder *,2,15 - 19,Males,"Employed, worked full-time",236
Albury,Murder *,2,15 - 19,Females,"Employed, worked full-time",113
Albury,Murder *,2,15 - 19,Persons,"Employed, worked full-time",348
Albury,Murder *,2,15 - 19,Males,"Employed, worked part-time",431
Albury,Murder *,2,15 - 19,Females,"Employed, worked part-time",641
Albury,Murder *,2,15 - 19,Persons,"Employed, worked part-time",1074


Observations: 1,860,310
Variables: 7
$ LGA                   <chr> "Albury", "Albury", "Albury", "Albury", "Albu...
$ Offence               <fct> Murder *, Murder *, Murder *, Murder *, Murde...
$ Crim_Count            <int> 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
$ Age                   <fct> 15 - 19, 15 - 19, 15 - 19, 15 - 19, 15 - 19, ...
$ Sex                   <fct> Males, Females, Persons, Males, Females, Pers...
$ `Labour force status` <fct> "Employed, worked full-time", "Employed, work...
$ pop_count             <int> 236, 113, 348, 431, 641, 1074, 27, 26, 59, 22...


In [172]:
out_filename = "joined_crime_labour.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `industry_employment`

In [158]:
head(input_data$crime)
head(input_data$pop_dens)

X1,LGA,Offence,Crim_Count
1,Albury,Murder *,2
2,Albury,Attempted murder,1
3,Albury,"Murder accessory, conspiracy",0
4,Albury,Manslaughter *,0
5,Albury,Domestic violence related assault,284
6,Albury,Non-domestic violence related assault,296


X1,LGA,Pop_Dens_km2
1,Albury,170.5
2,Armidale Regional,3.5
3,Ballina,88.7
4,Balranald,0.1
5,Bathurst Regional,11.1
6,Bega Valley,5.4


In [161]:
joined <- input_data$crime %>%
    select(-X1) %>%
    left_join(input_data$pop_dens %>% select(-X1))

head(joined)
glimpse(joined)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,Offence,Crim_Count,Pop_Dens_km2
Albury,Murder *,2,170.5
Albury,Attempted murder,1,170.5
Albury,"Murder accessory, conspiracy",0,170.5
Albury,Manslaughter *,0,170.5
Albury,Domestic violence related assault,284,170.5
Albury,Non-domestic violence related assault,296,170.5


Observations: 8,060
Variables: 4
$ LGA          <chr> "Albury", "Albury", "Albury", "Albury", "Albury", "Alb...
$ Offence      <fct> "Murder *", "Attempted murder", "Murder accessory, con...
$ Crim_Count   <int> 2, 1, 0, 0, 284, 296, 25, 47, 68, 2, 12, 3, 6, 0, 315,...
$ Pop_Dens_km2 <dbl> 170.5, 170.5, 170.5, 170.5, 170.5, 170.5, 170.5, 170.5...


In [162]:
out_filename = "joined_crime_popdensity.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `seifa`

In [163]:
head(input_data$crime)
head(input_data$seifa)

X1,LGA,Offence,Crim_Count
1,Albury,Murder *,2
2,Albury,Attempted murder,1
3,Albury,"Murder accessory, conspiracy",0
4,Albury,Manslaughter *,0
5,Albury,Domestic violence related assault,284
6,Albury,Non-domestic violence related assault,296


X1,LGA,Value
1,Albury,964
2,Armidale Dumaresq,954
3,Ashfield,962
4,Auburn,931
5,Ballina,986
6,Balranald,969


In [164]:
joined <- input_data$crime %>%
    select(-X1) %>%
    left_join(input_data$seifa %>% select(-X1)) %>%
    rename(seifa = Value)

head(joined)
glimpse(joined)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,Offence,Crim_Count,seifa
Albury,Murder *,2,964
Albury,Attempted murder,1,964
Albury,"Murder accessory, conspiracy",0,964
Albury,Manslaughter *,0,964
Albury,Domestic violence related assault,284,964
Albury,Non-domestic violence related assault,296,964


Observations: 8,122
Variables: 4
$ LGA        <chr> "Albury", "Albury", "Albury", "Albury", "Albury", "Albur...
$ Offence    <fct> "Murder *", "Attempted murder", "Murder accessory, consp...
$ Crim_Count <int> 2, 1, 0, 0, 284, 296, 25, 47, 68, 2, 12, 3, 6, 0, 315, 1...
$ seifa      <int> 964, 964, 964, 964, 964, 964, 964, 964, 964, 964, 964, 9...


In [165]:
out_filename = "joined_crime_seifa.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Joining `crime` and `transport`

In [166]:
head(input_data$crime)
head(input_data$transport)

X1,LGA,Offence,Crim_Count
1,Albury,Murder *,2
2,Albury,Attempted murder,1
3,Albury,"Murder accessory, conspiracy",0
4,Albury,Manslaughter *,0
5,Albury,Domestic violence related assault,284
6,Albury,Non-domestic violence related assault,296


LGA,route_type,num_service
Albury,106,131
Albury,204,14
Albury,700,1624
Armidale Regional,106,62
Armidale Regional,204,10
Armidale Regional,700,113


In [168]:
joined <- input_data$crime %>%
    select(-X1) %>%
    left_join(input_data$transport)

head(joined)
glimpse(joined)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,Offence,Crim_Count,route_type,num_service
Albury,Murder *,2,106,131
Albury,Murder *,2,204,14
Albury,Murder *,2,700,1624
Albury,Attempted murder,1,106,131
Albury,Attempted murder,1,204,14
Albury,Attempted murder,1,700,1624


Observations: 22,010
Variables: 5
$ LGA         <chr> "Albury", "Albury", "Albury", "Albury", "Albury", "Albu...
$ Offence     <fct> "Murder *", "Murder *", "Murder *", "Attempted murder",...
$ Crim_Count  <int> 2, 2, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 284, 284, 284, 296,...
$ route_type  <fct> 106, 204, 700, 106, 204, 700, 106, 204, 700, 106, 204, ...
$ num_service <dbl> 131, 14, 1624, 131, 14, 1624, 131, 14, 1624, 131, 14, 1...


In [169]:
out_filename = "joined_crime_transport.csv"
write.csv(joined, paste(output_path, out_filename, sep = "/"), row.names = FALSE)

# Create Aggregated Data Set

In order to have all the variables in one data set, eat data set has to be aggregated at the LGA level. This means losing information on `age` and `sex` etc.

In [182]:
map(input_data, head)

$age
# A tibble: 6 x 8
  Age    Sex    `Labour force sta~ State   LGA_2016 LGA      `Census year` Value
  <fct>  <fct>  <fct>              <chr>      <dbl> <fct>            <dbl> <int>
1 15 - ~ Males  Employed, worked ~ New So~    10050 Albury            2016   236
2 15 - ~ Femal~ Employed, worked ~ New So~    10050 Albury            2016   113
3 15 - ~ Perso~ Employed, worked ~ New So~    10050 Albury            2016   348
4 15 - ~ Males  Employed, worked ~ New So~    10130 Armidal~          2016   105
5 15 - ~ Femal~ Employed, worked ~ New So~    10130 Armidal~          2016    58
6 15 - ~ Perso~ Employed, worked ~ New So~    10130 Armidal~          2016   160

$country_of_birth
# A tibble: 6 x 7
  birthplace_of_pe~ census_year lga_2016 LGA   value yarrp_2016 year_of_arrival~
  <fct>                   <dbl>    <dbl> <fct> <dbl> <chr>      <fct>           
1 New Zealand              2016    10050 Albu~     7 2          Arrived 1946 - ~
2 New Zealand              2016    10050 Albu~    22 3          Arrived 1956 - ~
3 New Zealand              2016    10050 Albu~    63 4          Arrived 1966 - ~
4 New Zealand              2016    10050 Albu~   113 5          Arrived 1976 - ~
5 New Zealand              2016    10050 Albu~    92 6          Arrived 1986 - ~
6 New Zealand              2016    10050 Albu~    87 7          Arrived 1996 - ~

$crime
# A tibble: 6 x 4
     X1 LGA    Offence                               Crim_Count
  <dbl> <fct>  <fct>                                      <int>
1     1 Albury Murder *                                       2
2     2 Albury Attempted murder                               1
3     3 Albury Murder accessory, conspiracy                   0
4     4 Albury Manslaughter *                                 0
5     5 Albury Domestic violence related assault            284
6     6 Albury Non-domestic violence related assault        296

$industry_employment
# A tibble: 6 x 7
  Age    Sex    `Industry of Employment` LGA_2016 LGA        `Census year` Value
  <fct>  <fct>  <fct>                       <dbl> <fct>              <dbl> <int>
1 15 - ~ Males  Agriculture, Forestry a~    10050 Albury              2016     6
2 15 - ~ Femal~ Agriculture, Forestry a~    10050 Albury              2016     4
3 15 - ~ Perso~ Agriculture, Forestry a~    10050 Albury              2016     6
4 15 - ~ Males  Agriculture, Forestry a~    10130 Armidale ~          2016    34
5 15 - ~ Femal~ Agriculture, Forestry a~    10130 Armidale ~          2016     9
6 15 - ~ Perso~ Agriculture, Forestry a~    10130 Armidale ~          2016    43

$labour
# A tibble: 6 x 7
  Age     Sex     `Labour force status` LGA_2016 LGA         `Census year` Value
  <fct>   <fct>   <fct>                    <dbl> <fct>               <dbl> <int>
1 15 - 19 Males   Employed, worked ful~    10050 Albury               2016   236
2 15 - 19 Females Employed, worked ful~    10050 Albury               2016   113
3 15 - 19 Persons Employed, worked ful~    10050 Albury               2016   348
4 15 - 19 Males   Employed, worked ful~    10130 Armidale R~          2016   105
5 15 - 19 Females Employed, worked ful~    10130 Armidale R~          2016    58
6 15 - 19 Persons Employed, worked ful~    10130 Armidale R~          2016   160

$pop_dens
# A tibble: 6 x 3
     X1 LGA               Pop_Dens_km2
  <dbl> <fct>                    <dbl>
1     1 Albury                   170. 
2     2 Armidale Regional          3.5
3     3 Ballina                   88.7
4     4 Balranald                  0.1
5     5 Bathurst Regional         11.1
6     6 Bega Valley                5.4

$seifa
# A tibble: 6 x 3
     X1 LGA               Value
  <dbl> <fct>             <int>
1     1 Albury              964
2     2 Armidale Dumaresq   954
3     3 Ashfield            962
4     4 Auburn              931
5     5 Ballina             986
6     6 Balranald           969

$transport
# A tibble: 6 x 3
  LGA               route_type num_service
  <fct>             <fct>            <dbl>
1 Al

In [192]:
value_cols <- list("Value", "value", "Crim_Count", "Value", "Value", "Pop_Dens_km2", "Value", "num_service")

In [193]:
names(value_cols) <- tbl_names
names(value_cols)

[1] "age"                 "country_of_birth"    "crime"              
[4] "industry_employment" "labour"              "pop_dens"           
[7] "seifa"               "transport"

Create a function to take the data and aggregate on the `LGA` column.

In [234]:
simplify = function(X, tbl_name, key_col, value_col){
    return(X %>% group_by(!!sym(key_col)) %>%
            summarise(!!tbl_name := sum(!!sym(value_col))))
}

In [188]:
simplified_data <- map(input_data)

Age,Sex,Labour force status,State,LGA_2016,LGA,Census year,Value
15 - 19,Males,"Employed, worked full-time",New South Wales,10050,Albury,2016,236
15 - 19,Females,"Employed, worked full-time",New South Wales,10050,Albury,2016,113
15 - 19,Persons,"Employed, worked full-time",New South Wales,10050,Albury,2016,348
15 - 19,Males,"Employed, worked full-time",New South Wales,10130,Armidale Regional,2016,105
15 - 19,Females,"Employed, worked full-time",New South Wales,10130,Armidale Regional,2016,58
15 - 19,Persons,"Employed, worked full-time",New South Wales,10130,Armidale Regional,2016,160


Create a list of lists to be used in `pmap` with `simplify`. The list names/positions are the same as the argument names/positions of the function.

In [242]:
input_list <- list(X = input_data,
                   tbl_name = tbl_names,
                   key_col = rep("LGA", length(input_data)),
                   value_col = value_cols)

Aggregate all data sets in one line.

In [243]:
simplified_data <- pmap(input_list, simplify)

In [244]:
map(simplified_data, head)

LGA,age
Albury,264198
Armidale Regional,149854
Ballina,213610
Balranald,11420
Bathurst Regional,210047
Bega Valley,167576
LGA,country_of_birth
Albury,13595
Armidale Regional,9054
Ballina,17109


The aggregations by LGA for some data sets don't give meaningful information. So it is better to process each data set manually by spreading them first to maintain some variables.

# Spread the Data

In [257]:
age_spread <- input_data$age %>%
    spread(key = Sex, value = Value) %>%
    group_by(LGA) %>%
    summarise(males = sum(Males), females = sum(Females), persons = sum(Persons))

head(age_spread)

LGA,males,females,persons
Albury,64518,67583,132097
Armidale Regional,36679,38229,74946
Ballina,51734,55062,106814
Balranald,3041,2669,5710
Bathurst Regional,52747,52257,105043
Bega Valley,41321,42491,83764


In [313]:
country_spread <- input_data$country_of_birth %>%
    group_by(LGA, birthplace_of_person) %>%
    summarise(value = sum(value)) %>%
    rename(birth_value = value)

head(country_spread)

LGA,birthplace_of_person,birth_value
Albury,New Zealand,2510
Albury,Papua New Guinea,99
Albury,Fiji,218
Albury,Tonga,24
Albury,England,4254
Albury,Northern Ireland,64


In [275]:
employment_spread <- input_data$industry_employment %>%
    group_by(LGA, `Industry of Employment`) %>%
    summarise(emp_value = sum(Value))

head(employment_spread)

LGA,Industry of Employment,emp_value
Albury,"Agriculture, Forestry and Fishing",1253
Albury,Mining,191
Albury,Manufacturing,8435
Albury,"Electricity, Gas, Water and Waste Services",713
Albury,Construction,8425
Albury,Wholesale Trade,2137


In [281]:
labour_spread <- input_data$labour %>%
    group_by(LGA, `Labour force status`) %>%
    summarise(labour_value = sum(Value))

head(labour_spread)

LGA,Labour force status,labour_value
Albury,"Employed, worked full-time",54383
Albury,"Employed, worked part-time",32346
Albury,"Employed, away from work",3331
Albury,"Employed, hours of work not stated",1567
Albury,"Unemployed, looking for full-time work",4019
Albury,"Unemployed, looking for part-time work",2711


In [281]:
labour_spread <- input_data$pop_dens %>%
    group_by(LGA, `Labour force status`) %>%
    summarise(labour_value = sum(Value))

head(labour_spread)

LGA,Labour force status,labour_value
Albury,"Employed, worked full-time",54383
Albury,"Employed, worked part-time",32346
Albury,"Employed, away from work",3331
Albury,"Employed, hours of work not stated",1567
Albury,"Unemployed, looking for full-time work",4019
Albury,"Unemployed, looking for part-time work",2711


# Joining the Aggregated Data

In [314]:
joined_agg <- input_data$crime %>%
    group_by(LGA) %>%
    summarise(crime = sum(Crim_Count)) %>%
    left_join(age_spread)

head(joined_agg)
glimpse(joined_agg)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining factors with different levels, coercing to character vector"

LGA,crime,males,females,persons
Albury,6233,64518,67583,132097
Armidale Regional,4028,36679,38229,74946
Ballina,2690,51734,55062,106814
Balranald,192,3041,2669,5710
Bathurst Regional,4027,52747,52257,105043
Bayside,13024,NA,NA,NA


Observations: 130
Variables: 5
$ LGA     <chr> "Albury", "Armidale Regional", "Ballina", "Balranald", "Bat...
$ crime   <int> 6233, 4028, 2690, 192, 4027, 13024, 2119, 798, 482, 45643, ...
$ males   <int> 64518, 36679, 51734, 3041, 52747, NA, 41321, 15436, 10614, ...
$ females <int> 67583, 38229, 55062, 2669, 52257, NA, 42491, 16022, 10340, ...
$ persons <int> 132097, 74946, 106814, 5710, 105043, NA, 83764, 31479, 2092...


In [315]:
joined_agg %<>%
    left_join(country_spread)

head(joined_agg)
glimpse(joined_agg)

Joining, by = "LGA"
Warning message:
"Column `LGA` joining character vector and factor, coercing into character vector"

LGA,crime,males,females,persons,birthplace_of_person,birth_value
Albury,6233,64518,67583,132097,New Zealand,2510
Albury,6233,64518,67583,132097,Papua New Guinea,99
Albury,6233,64518,67583,132097,Fiji,218
Albury,6233,64518,67583,132097,Tonga,24
Albury,6233,64518,67583,132097,England,4254
Albury,6233,64518,67583,132097,Northern Ireland,64


Observations: 8,072
Variables: 7
$ LGA                  <chr> "Albury", "Albury", "Albury", "Albury", "Albur...
$ crime                <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, 6233...
$ males                <int> 64518, 64518, 64518, 64518, 64518, 64518, 6451...
$ females              <int> 67583, 67583, 67583, 67583, 67583, 67583, 6758...
$ persons              <int> 132097, 132097, 132097, 132097, 132097, 132097...
$ birthplace_of_person <fct> "New Zealand", "Papua New Guinea", "Fiji", "To...
$ birth_value          <dbl> 2510, 99, 218, 24, 4254, 64, 800, 213, 325, 25...


In [316]:
joined_agg %<>%
    left_join(employment_spread, by = "LGA")

head(joined_agg)
glimpse(joined_agg)

Warning message:
"Column `LGA` joining character vector and factor, coercing into character vector"

LGA,crime,males,females,persons,birthplace_of_person,birth_value,Industry of Employment,emp_value
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253
Albury,6233,64518,67583,132097,New Zealand,2510,Mining,191
Albury,6233,64518,67583,132097,New Zealand,2510,Manufacturing,8435
Albury,6233,64518,67583,132097,New Zealand,2510,"Electricity, Gas, Water and Waste Services",713
Albury,6233,64518,67583,132097,New Zealand,2510,Construction,8425
Albury,6233,64518,67583,132097,New Zealand,2510,Wholesale Trade,2137


Observations: 161,345
Variables: 9
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ crime                    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ males                    <int> 64518, 64518, 64518, 64518, 64518, 64518, ...
$ females                  <int> 67583, 67583, 67583, 67583, 67583, 67583, ...
$ persons                  <int> 132097, 132097, 132097, 132097, 132097, 13...
$ birthplace_of_person     <fct> New Zealand, New Zealand, New Zealand, New...
$ birth_value              <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, ...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Mini...
$ emp_value                <int> 1253, 191, 8435, 713, 8425, 2137, 10520, 7...


In [317]:
joined_agg %<>%
    left_join(labour_spread, by = "LGA")

head(joined_agg)
glimpse(joined_agg)

Warning message:
"Column `LGA` joining character vector and factor, coercing into character vector"

LGA,crime,males,females,persons,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, away from work",3331
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, hours of work not stated",1567
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Unemployed, looking for full-time work",4019
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Unemployed, looking for part-time work",2711


Observations: 1,290,725
Variables: 11
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ crime                    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ males                    <int> 64518, 64518, 64518, 64518, 64518, 64518, ...
$ females                  <int> 67583, 67583, 67583, 67583, 67583, 67583, ...
$ persons                  <int> 132097, 132097, 132097, 132097, 132097, 13...
$ birthplace_of_person     <fct> New Zealand, New Zealand, New Zealand, New...
$ birth_value              <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, ...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ emp_value                <int> 1253, 1253, 1253, 1253, 1253, 1253, 1253, ...
$ `Labour force status`    <fct> "Employed, worked full-time", "Employed, w...
$ labour_value             <int> 54383, 32346, 3331, 1567, 4019, 2711, 5539...


In [318]:
joined_agg %<>%
    left_join(input_data$pop_dens %>% select(-X1), by = "LGA")

head(joined_agg)
glimpse(joined_agg)

Warning message:
"Column `LGA` joining character vector and factor, coercing into character vector"

LGA,crime,males,females,persons,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, away from work",3331,170.5
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, hours of work not stated",1567,170.5
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Unemployed, looking for full-time work",4019,170.5
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Unemployed, looking for part-time work",2711,170.5


Observations: 1,290,725
Variables: 12
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ crime                    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ males                    <int> 64518, 64518, 64518, 64518, 64518, 64518, ...
$ females                  <int> 67583, 67583, 67583, 67583, 67583, 67583, ...
$ persons                  <int> 132097, 132097, 132097, 132097, 132097, 13...
$ birthplace_of_person     <fct> New Zealand, New Zealand, New Zealand, New...
$ birth_value              <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, ...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ emp_value                <int> 1253, 1253, 1253, 1253, 1253, 1253, 1253, ...
$ `Labour force status`    <fct> "Employed, worked full-time", "Employed, w...
$ labour_value             <int> 54383, 32346, 3331, 1567, 4019, 2711, 5539...
$ Pop_Dens_km2             <dbl> 170.5, 170.5, 170.5, 170.5, 170.5, 170.5, ...


In [319]:
joined_agg %<>%
    left_join(input_data$seifa %>% select(-X1) %>% rename(seifa_value = Value), by = "LGA")

head(joined_agg)
glimpse(joined_agg)

Warning message:
"Column `LGA` joining character vector and factor, coercing into character vector"

LGA,crime,males,females,persons,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2,seifa_value
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, away from work",3331,170.5,964
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, hours of work not stated",1567,170.5,964
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Unemployed, looking for full-time work",4019,170.5,964
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Unemployed, looking for part-time work",2711,170.5,964


Observations: 1,314,885
Variables: 13
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ crime                    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ males                    <int> 64518, 64518, 64518, 64518, 64518, 64518, ...
$ females                  <int> 67583, 67583, 67583, 67583, 67583, 67583, ...
$ persons                  <int> 132097, 132097, 132097, 132097, 132097, 13...
$ birthplace_of_person     <fct> New Zealand, New Zealand, New Zealand, New...
$ birth_value              <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, ...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ emp_value                <int> 1253, 1253, 1253, 1253, 1253, 1253, 1253, ...
$ `Labour force status`    <fct> "Employed, worked full-time", "Employed, w...
$ labour_value             <int> 54383, 32346, 3331, 1567, 4019, 2711, 5539...
$ Pop_Dens_km2             <dbl> 170.5, 170.5, 170.5, 170.5, 170.5, 170.5, ...
$ seifa_value 

In [320]:
joined_agg %<>%
    left_join(input_data$transport, by = "LGA")

head(joined_agg)
glimpse(joined_agg)

Warning message:
"Column `LGA` joining character vector and factor, coercing into character vector"

LGA,crime,males,females,persons,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2,seifa_value,route_type,num_service
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,106,131
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,204,14
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,700,1624
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,106,131
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,204,14
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,700,1624


Observations: 4,510,890
Variables: 15
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ crime                    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ males                    <int> 64518, 64518, 64518, 64518, 64518, 64518, ...
$ females                  <int> 67583, 67583, 67583, 67583, 67583, 67583, ...
$ persons                  <int> 132097, 132097, 132097, 132097, 132097, 13...
$ birthplace_of_person     <fct> New Zealand, New Zealand, New Zealand, New...
$ birth_value              <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, ...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ emp_value                <int> 1253, 1253, 1253, 1253, 1253, 1253, 1253, ...
$ `Labour force status`    <fct> "Employed, worked full-time", "Employed, w...
$ labour_value             <int> 54383, 54383, 54383, 32346, 32346, 32346, ...
$ Pop_Dens_km2             <dbl> 170.5, 170.5, 170.5, 170.5, 170.5, 170.5, ...
$ seifa_value 

# Adding Crime Rate

Join `population`

In [323]:
joined_agg %<>%
    mutate(crime_rate = crime / (males + females))

head(joined_agg)
glimpse(joined_agg)

LGA,crime,males,females,persons,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2,seifa_value,route_type,num_service,crime_rate
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,106,131,0.04718359
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,204,14,0.04718359
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,700,1624,0.04718359
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,106,131,0.04718359
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,204,14,0.04718359
Albury,6233,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,700,1624,0.04718359


Observations: 4,510,890
Variables: 16
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ crime                    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ males                    <int> 64518, 64518, 64518, 64518, 64518, 64518, ...
$ females                  <int> 67583, 67583, 67583, 67583, 67583, 67583, ...
$ persons                  <int> 132097, 132097, 132097, 132097, 132097, 13...
$ birthplace_of_person     <fct> New Zealand, New Zealand, New Zealand, New...
$ birth_value              <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, ...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ emp_value                <int> 1253, 1253, 1253, 1253, 1253, 1253, 1253, ...
$ `Labour force status`    <fct> "Employed, worked full-time", "Employed, w...
$ labour_value             <int> 54383, 54383, 54383, 32346, 32346, 32346, ...
$ Pop_Dens_km2             <dbl> 170.5, 170.5, 170.5, 170.5, 170.5, 170.5, ...
$ seifa_value 

Reorder columns

In [324]:
ordered_cols <- c("LGA","crime","crime_rate","males","females","persons","birthplace_of_person","birth_value","Industry of Employment","emp_value","Labour force status","labour_value","Pop_Dens_km2","seifa_value","route_type","num_service")

In [325]:
joined_agg %<>%
    select(ordered_cols)

head(joined_agg)
glimpse(joined_agg)

LGA,crime,crime_rate,males,females,persons,birthplace_of_person,birth_value,Industry of Employment,emp_value,Labour force status,labour_value,Pop_Dens_km2,seifa_value,route_type,num_service
Albury,6233,0.04718359,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,106,131
Albury,6233,0.04718359,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,204,14
Albury,6233,0.04718359,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked full-time",54383,170.5,964,700,1624
Albury,6233,0.04718359,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,106,131
Albury,6233,0.04718359,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,204,14
Albury,6233,0.04718359,64518,67583,132097,New Zealand,2510,"Agriculture, Forestry and Fishing",1253,"Employed, worked part-time",32346,170.5,964,700,1624


Observations: 4,510,890
Variables: 16
$ LGA                      <chr> "Albury", "Albury", "Albury", "Albury", "A...
$ crime                    <int> 6233, 6233, 6233, 6233, 6233, 6233, 6233, ...
$ crime_rate               <dbl> 0.04718359, 0.04718359, 0.04718359, 0.0471...
$ males                    <int> 64518, 64518, 64518, 64518, 64518, 64518, ...
$ females                  <int> 67583, 67583, 67583, 67583, 67583, 67583, ...
$ persons                  <int> 132097, 132097, 132097, 132097, 132097, 13...
$ birthplace_of_person     <fct> New Zealand, New Zealand, New Zealand, New...
$ birth_value              <dbl> 2510, 2510, 2510, 2510, 2510, 2510, 2510, ...
$ `Industry of Employment` <fct> "Agriculture, Forestry and Fishing", "Agri...
$ emp_value                <int> 1253, 1253, 1253, 1253, 1253, 1253, 1253, ...
$ `Labour force status`    <fct> "Employed, worked full-time", "Employed, w...
$ labour_value             <int> 54383, 54383, 54383, 32346, 32346, 32346, ...
$ Pop_Dens_km2

In [326]:
write.csv(joined, paste(output_path, "unified2.csv", sep = "/"), row.names = FALSE)

In [327]:
save.image()